In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Input
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

import numpy as np

from tensorflow.keras.optimizers import Adam

import numpy as np

In [2]:
def preprocess_input_grayscale(img):
    return np.repeat(img, 3, axis=-1)

In [3]:
train_datagen = ImageDataGenerator(
    # preprocessing_function=preprocess_input_grayscale,
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='wrap',
)

train_generator = train_datagen.flow_from_directory(
    'Dataset_train',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
)

val_datagen = ImageDataGenerator(
    # preprocessing_function=preprocess_input_grayscale,
    rescale=1./255
)

val_generator = val_datagen.flow_from_directory(
    'Dataset_testing',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

Found 4444 images belonging to 8 classes.
Found 1114 images belonging to 8 classes.


In [4]:
num_classes = len(train_generator.class_indices)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(224,224,3)),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax') #Output
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'precision', 'recall'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     1,311,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,227,912 (16.13 MB)

 Trainable params: 1,969,928 (7.51 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [5]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=5,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    verbose=1
)

c:\Users\rizfi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 687ms/step - accuracy: 0.1985 - loss: 2.2209 - precision: 0.2109 - recall: 0.0304

c:\Users\rizfi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


139/139 ━━━━━━━━━━━━━━━━━━━━ 120s 817ms/step - accuracy: 0.1987 - loss: 2.2198 - precision: 0.2111 - recall: 0.0303 - val_accuracy: 0.2738 - val_loss: 1.8696 - val_precision: 0.8000 - val_recall: 0.0036
Epoch 2/5
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00


c:\Users\rizfi\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


Epoch 3/5
139/139 ━━━━━━━━━━━━━━━━━━━━ 108s 780ms/step - accuracy: 0.2907 - loss: 1.8638 - precision: 0.4776 - recall: 0.0310 - val_accuracy: 0.2684 - val_loss: 1.8584 - val_precision: 0.6750 - val_recall: 0.0485
Epoch 4/5
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 385us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 5/5
139/139 ━━━━━━━━━━━━━━━━━━━━ 111s 796ms/step - accuracy: 0.3186 - loss: 1.7799 - precision: 0.5168 - recall: 0.0565 - val_accuracy: 0.3456 - val_loss: 1.7579 - val_precision: 0.7241 - val_recall: 0.0189


In [6]:
# Mendapatkan nilai akurasi untuk setiap epoch dari history
train_accuracy = history.history['accuracy']  # Akurasi pada data training
val_accuracy = history.history['val_accuracy']  # Akurasi pada data validasi

# Mencetak akurasi di setiap epoch
print("Training Accuracy per Epoch:")
for epoch, acc in enumerate(train_accuracy, 1):
    print(f"Epoch {epoch}: {acc:.4f}")

print("\nValidation Accuracy per Epoch:")
for epoch, val_acc in enumerate(val_accuracy, 1):
    print(f"Epoch {epoch}: {val_acc:.4f}")

Training Accuracy per Epoch:
Epoch 1: 0.2270
Epoch 2: 0.0000
Epoch 3: 0.3031
Epoch 4: 0.0000
Epoch 5: 0.3209

Validation Accuracy per Epoch:
Epoch 1: 0.2738
Epoch 2: 0.2684
Epoch 3: 0.3456


In [7]:
for layer in base_model.layers[:-20]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

fine_tune_history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=15,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    verbose=1
)

Epoch 1/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 352s 2s/step - accuracy: 0.2558 - loss: 1.9132 - val_accuracy: 0.3573 - val_loss: 1.7211
Epoch 2/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 326s 2s/step - accuracy: 0.3000 - loss: 1.8142 - val_accuracy: 0.3555 - val_loss: 1.7067
Epoch 4/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 320s 2s/step - accuracy: 0.3360 - loss: 1.7667 - val_accuracy: 0.3600 - val_loss: 1.7041
Epoch 6/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 512us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 315s 2s/step - accuracy: 0.3468 - loss: 1.7251 - val_accuracy: 0.3582 - val_loss: 1.7204
Epoch 8/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 318s 2s/step - accuracy: 0.3606 - loss: 1.6866 - val_accuracy:

In [8]:
# Mendapatkan nilai akurasi untuk setiap epoch dari history
train_accuracy = fine_tune_history.history['accuracy']  # Akurasi pada data training
val_accuracy = fine_tune_history.history['val_accuracy']  # Akurasi pada data validasi
# Mencetak akurasi di setiap epoch
print("Training Accuracy per Epoch:")
for epoch, acc in enumerate(train_accuracy, 1):
    print(f"Epoch {epoch}: {acc:.4f}")

print("\nValidation Accuracy per Epoch:")
for epoch, val_acc in enumerate(val_accuracy, 1):
    print(f"Epoch {epoch}: {val_acc:.4f}")

Training Accuracy per Epoch:
Epoch 1: 0.2712
Epoch 2: 0.0000
Epoch 3: 0.3078
Epoch 4: 0.0000
Epoch 5: 0.3326
Epoch 6: 0.0000
Epoch 7: 0.3445
Epoch 8: 0.0000
Epoch 9: 0.3576
Epoch 10: 0.0000
Epoch 11: 0.3672
Epoch 12: 0.0000
Epoch 13: 0.3715
Epoch 14: 0.0000
Epoch 15: 0.3834

Validation Accuracy per Epoch:
Epoch 1: 0.3573
Epoch 2: 0.3555
Epoch 3: 0.3600
Epoch 4: 0.3582
Epoch 5: 0.3609
Epoch 6: 0.3725
Epoch 7: 0.3824
Epoch 8: 0.3995


In [14]:
test_images, test_labels = next(val_generator)

test_images = test_images[:10]
test_labels = test_labels[:10]

predictions = model.predict(test_images)

predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(test_labels, axis=1)

# Tampilkan hasil prediksi untuk 10 data
for i in range(10):
    print(f"Predicted: {predicted_classes[i]}, True: {true_classes[i]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted: 6, True: 0
Predicted: 0, True: 7
Predicted: 4, True: 5
Predicted: 4, True: 7
Predicted: 4, True: 1
Predicted: 0, True: 0
Predicted: 6, True: 4
Predicted: 5, True: 4
Predicted: 6, True: 5
Predicted: 4, True: 6
